# 흐름
- XGBOOST 시계열 변환
- DeadCut : 연속으로 14일이 0인 행 드롭 (23% 드롭)
- 위치기반 칼럼 추가

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import argparse
import pandas as pd
from tqdm import tqdm
from datetime import timedelta

# -------------------------------
# Hyper-parameters
# -------------------------------
LAGS = 28          # 과거 28일
HORIZON = 7        # 예측 7일
# CatBoost에 넘길 범주형 컬럼(문자형 + 범주 취급할 수치)
CAT_FEATURES = ['영업장명_메뉴명', 'dow', 'month']


base_path = '/content/drive/MyDrive/data'
train_path=base_path +"/train/train.csv"
test_dir  =base_path+"/test"        # TEST00,01… 폴더
out_path  =base_path+"/submission.csv"

args = argparse.Namespace(train=train_path, test=test_dir, out=out_path)

def read_data(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    # 컬럼 존재 확인
    required = {'영업일자', '영업장명_메뉴명', '매출수량'}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"입력 파일에 필요한 컬럼이 없습니다: {missing}")
    # 타입 캐스팅
    df['영업일자'] = pd.to_datetime(df['영업일자'])
    # 매출수량 결측은 0으로 (필요 시 조정)
    if df['매출수량'].isna().any():
        df['매출수량'] = df['매출수량'].fillna(0)
    return df

def add_calendar_feats(df: pd.DataFrame) -> pd.DataFrame:
    """요일, 월 등 캘린더 변수 추가"""
    df = df.copy()
    df['dow'] = df['영업일자'].dt.dayofweek  # 0=월
    df['month'] = df['영업일자'].dt.month
    return df

def ensure_daily_continuity(grp: pd.DataFrame) -> pd.DataFrame:
    """
    그룹(영업장명_메뉴명) 단위로 일자 연속성 확보.
    빠진 날짜는 매출수량=0으로 채움. 필요시 다른 보간법으로 교체.
    """
    g = grp.set_index('영업일자').sort_index()
    # 전체 기간에 대해 일 단위 리샘플
    g = g.asfreq('D')
    # 문자/카테고리 컬럼 보전
    g['영업장명_메뉴명'] = g['영업장명_메뉴명'].ffill().bfill()
    # 매출수량 결측은 0
    g['매출수량'] = g['매출수량'].fillna(0)
    g = g.reset_index().rename(columns={'index': '영업일자'})
    return g

In [13]:
# -------------------------------
# 2. 슬라이딩 윈도우 생성
# -------------------------------
def build_lag_samples(
    df: pd.DataFrame,
    lags: int = LAGS,
    horizon: int = HORIZON,
    train_mode: bool = True
):
    """
    Train 모드:
        각 그룹별로 (lags -> horizon) 윈도우를 모든 가능한 위치에서 생성.
        반환: X(DataFrame), y(DataFrame), meta(ref_date 포함)

    Test 모드:
        각 그룹별로 마지막 28일만 사용하여 1행 생성.
        반환: X_test(DataFrame), keys(list[(key, 예측시작일)])
    """
    X_rows, y_rows, keys = [], [], []

    for key, grp in tqdm(df.groupby('영업장명_메뉴명'), desc='window'):
        g = ensure_daily_continuity(grp)
        sales = g['매출수량'].values
        dates = g['영업일자']

        if len(sales) < lags:
            # 라그를 만들 수 없으면 스킵 (혹은 패딩)
            continue

        if train_mode:
            # 전체 기간에서 슬라이딩
            for i in range(lags, len(sales) - horizon):
                lag_block = sales[i - lags:i][::-1]     # 최신값이 lag_1
                target = sales[i:i + horizon]          # t+1 ... t+H
                ref_date = dates.iloc[i]               # 기준 시점(라그의 바로 다음 날)

                X_row = {
                    '영업장명_메뉴명': key,
                    'dow': int(ref_date.dayofweek),
                    'month': int(ref_date.month),
                    'ref_date': ref_date,              # 검증 split용 (학습시 drop)
                }
                for l in range(1, lags + 1):
                    X_row[f'lag_{l}'] = float(lag_block[l - 1])

                X_rows.append(X_row)
                y_rows.append(target)
                keys.append((key, ref_date))
        else:
            # 마지막 28일로 1행 생성 → 예측 시작일은 마지막 관측일 + 1
            lag_block = sales[-lags:][::-1]
            last_date = dates.iloc[-1]
            X_row = {
                '영업장명_메뉴명': key,
                'dow': int(last_date.dayofweek),
                'month': int(last_date.month),
            }
            for l in range(1, lags + 1):
                X_row[f'lag_{l}'] = float(lag_block[l - 1])

            X_rows.append(X_row)
            keys.append((key, last_date + timedelta(days=1)))

    X = pd.DataFrame(X_rows)

    if train_mode:
        y_cols = [f't+{h}' for h in range(1, horizon + 1)]
        y = pd.DataFrame(y_rows, columns=y_cols)
        # ref_date가 없는 행은 제거(안전장치)
        assert 'ref_date' in X.columns
        return X, y
    else:
        return X, keys


In [14]:
train_df = add_calendar_feats(read_data(args.train))
X_train, y_train = build_lag_samples(train_df, train_mode=True)

window: 100%|██████████| 193/193 [00:03<00:00, 61.00it/s]


In [15]:
X_train

,영업장명_메뉴명,dow,month,ref_date,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,...,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28
0,느티나무 셀프BBQ_1인 수저세트,6,1,2023-01-29,0.0,8.0,0.0,0.0,2.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,느티나무 셀프BBQ_1인 수저세트,0,1,2023-01-30,8.0,0.0,8.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,느티나무 셀프BBQ_1인 수저세트,1,1,2023-01-31,0.0,8.0,0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,느티나무 셀프BBQ_1인 수저세트,2,2,2023-02-01,4.0,0.0,8.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,느티나무 셀프BBQ_1인 수저세트,3,2,2023-02-02,6.0,4.0,0.0,8.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95916,화담숲카페_현미뻥스크림,1,6,2024-06-04,0.0,65.0,79.0,30.0,14.0,28.0,...,18.0,33.0,29.0,0.0,65.0,30.0,19.0,29.0,17.0,1.0
95917,화담숲카페_현미뻥스크림,2,6,2024-06-05,25.0,0.0,65.0,79.0,30.0,14.0,...,14.0,18.0,33.0,29.0,0.0,65.0,30.0,19.0,29.0,17.0
95918,화담숲카페_현미뻥스크림,3,6,2024-06-06,16.0,25.0,0.0,65.0,79.0,30.0,...,97.0,14.0,18.0,33.0,29.0,0.0,65.0,30.0,19.0,29.0
95919,화담숲카페_현미뻥스크림,4,6,2024-06-07,145.0,16.0,25.0,0.0,65.0,79.0,...,57.0,97.0,14.0,18.0,33.0,29.0,0.0,65.0,30.0,19.0


In [16]:
y_train

,t+1,t+2,t+3,t+4,t+5,t+6,t+7
0,8,0,4,6,2,0,2
1,0,4,6,2,0,2,2
2,4,6,2,0,2,2,4
3,6,2,0,2,2,4,5
4,2,0,2,2,4,5,0
...,...,...,...,...,...,...,...
95916,25,16,145,60,25,43,0
95917,16,145,60,25,43,0,12
95918,145,60,25,43,0,12,10
95919,60,25,43,0,12,10,14


In [17]:
import itertools, numpy as np
lag_cols = [f'lag_{i}' for i in range(1, 29)]  # lag_1 … lag_28

def max_zero_run(arr):
    runs = (len(list(g)) for v, g in itertools.groupby(arr) if v == 0.0)
    return max(runs, default=0)

# 연속 0 길이
X_train['max_zero_run'] = X_train[lag_cols].apply(max_zero_run, axis=1)

# 1) 연속 0 길이 계산 (이미 max_zero_run 컬럼 존재한다고 가정)
THR = 14                     # cut 기준
mask_keep = X_train['max_zero_run'] < THR

print(f"드롭 비율: {(~mask_keep).mean():.1%}")  # ≈ 18 %

# 2) 학습·튜닝용 데이터
X_fit = X_train.loc[mask_keep].drop(['max_zero_run'], axis=1)
y_fit = y_train.loc[mask_keep]


드롭 비율: 23.0%


In [18]:
X_fit[['영업장명', '메뉴명']] = X_fit['영업장명_메뉴명'].str.split('_', n=1, expand=True)
store_xy = {
    "담하"           : (620, 350),   # 기준점
    "미라시아"       : (620, 350),   # 동일 지점
    "느티나무 셀프BBQ": (500, 235),   # 북서쪽 165 px
    "포레스트릿"      : (770, 260),   # 북동쪽 175 px
    "연회장"          : (840, 380),   # 동-남쪽 220 px
    "카페테리아"      : (345, 300),   # 서쪽 275 px
    "화담숲주막"      : (920, 150),   # 북동쪽 360 px
    "화담숲카페"      : (920, 150),   # 주막과 동일 지점
    "연회장"        : (900, 350),    # 동쪽 평지
    "라그로타": (300, 300)
}
X_fit['x'] = X_fit['영업장명'].map(lambda s: store_xy[s][0])
X_fit['y'] = X_fit['영업장명'].map(lambda s: store_xy[s][1])
X_fit = X_fit.drop(columns=['영업장명', '메뉴명'])

In [26]:
X_fit

,영업장명_메뉴명,dow,month,ref_date,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,...,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28,x,y
3,느티나무 셀프BBQ_1인 수저세트,2,2,2023-02-01,4.0,0.0,8.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500,235
4,느티나무 셀프BBQ_1인 수저세트,3,2,2023-02-02,6.0,4.0,0.0,8.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500,235
5,느티나무 셀프BBQ_1인 수저세트,4,2,2023-02-03,2.0,6.0,4.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500,235
6,느티나무 셀프BBQ_1인 수저세트,5,2,2023-02-04,0.0,2.0,6.0,4.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500,235
7,느티나무 셀프BBQ_1인 수저세트,6,2,2023-02-05,2.0,0.0,2.0,6.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500,235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95916,화담숲카페_현미뻥스크림,1,6,2024-06-04,0.0,65.0,79.0,30.0,14.0,28.0,...,29.0,0.0,65.0,30.0,19.0,29.0,17.0,1.0,920,150
95917,화담숲카페_현미뻥스크림,2,6,2024-06-05,25.0,0.0,65.0,79.0,30.0,14.0,...,33.0,29.0,0.0,65.0,30.0,19.0,29.0,17.0,920,150
95918,화담숲카페_현미뻥스크림,3,6,2024-06-06,16.0,25.0,0.0,65.0,79.0,30.0,...,18.0,33.0,29.0,0.0,65.0,30.0,19.0,29.0,920,150
95919,화담숲카페_현미뻥스크림,4,6,2024-06-07,145.0,16.0,25.0,0.0,65.0,79.0,...,14.0,18.0,33.0,29.0,0.0,65.0,30.0,19.0,920,150


In [19]:
from typing import List, Tuple
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

# ------------------------------------------------------------------
# 1. 시간 기준 검증 분할 (그대로 재사용)
# ------------------------------------------------------------------
def time_based_split(X: pd.DataFrame,
                     y: pd.DataFrame,
                     valid_days: int = 14) -> Tuple[List[int], List[int]]:
    """ref_date 기준으로 최근 valid_days 일을 검증 세트로 사용"""
    cutoff = X['ref_date'].max() - pd.Timedelta(days=valid_days)
    train_idx = X[X['ref_date'] <= cutoff].index.tolist()
    val_idx   = X[X['ref_date'] >  cutoff].index.tolist()
    return train_idx, val_idx

# ------------------------------------------------------------------
# 2. XGBoost 학습 함수
# ------------------------------------------------------------------
def train_xgboost(
    X: pd.DataFrame,
    y: pd.DataFrame,
    gpu: bool = True,
):
    # ── 2-1. 검증 분할 ────────────────────────────────────────────
    train_idx, val_idx = time_based_split(X, y, valid_days=14)

    # ref_date 제거
    X_train = X.loc[train_idx].drop(columns=['ref_date'])
    X_val   = X.loc[val_idx]  .drop(columns=['ref_date'])
    y_train = y.loc[train_idx]
    y_val   = y.loc[val_idx]

    # ── 2-2. 전처리: 범주형 원-핫 + 수치형 패스 ─────────────────
    cat_cols  = ['dow', 'month','영업장명_메뉴명'] #'영업장명_메뉴명', '영업장명','메뉴명'
    num_cols  = [c for c in X_train.columns if c.startswith('lag_')]

    preproc = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
            ('num', 'passthrough',                   num_cols)
        ]
    )



    # ── 2-3. XGBRegressor (개별 타깃 7개를 한꺼번에) ─────────────
    base_model = XGBRegressor(
        n_estimators      = 2000,
        learning_rate     = 0.05,
        max_depth         = 6,
        subsample         = 0.8,
        colsample_bytree  = 0.8,
        objective         = 'reg:squarederror',
        random_state      = 42,
        tree_method       = 'gpu_hist' if gpu else 'hist',
        predictor        = 'gpu_predictor' ,
        # early_stopping_rounds 는 MultiOutputRegressor 안에서
        # 자동 지원이 안 되므로 일단 제외, 아래 manual callback 참고
    )

    model = Pipeline(steps=[
        ('prep', preproc),
        ('xgb',  MultiOutputRegressor(base_model, n_jobs=-1))
    ])

    # ── 2-4. 학습 (간단 버전: 일괄 학습) ─────────────────────────
    model.fit(X_train, y_train)

    # ── 2-5. 검증 RMSE 확인 (optional) ─────────────────────────
    preds = model.predict(X_val)
    rmse  = ((preds - y_val.values) ** 2).mean(axis=0) ** 0.5
    print("[Validation] RMSE per horizon:", rmse.round(2))

    return model


In [20]:
model = train_xgboost(X_fit, y_fit)

/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:1429: UserWarning: [14:23:35] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  return booster.num_features()


[Validation] RMSE per horizon: [17.   17.41 18.37 17.93 17.83 17.56 17.45]


In [30]:
def predict_test(
    model,
    X_test: pd.DataFrame,
    keys,
    test_prefix: str | None = None,
    horizon: int = HORIZON
) -> pd.DataFrame:
    """
    model : sklearn Pipeline (prep + MultiOutputRegressor(XGBRegressor))
    X_test: build_lag_samples(...) 결과 DataFrame (ref_date 컬럼 없음)
    """
    # ① CatBoost Pool 제거 → DataFrame 그대로 전달
    preds = model.predict(X_test)           # shape = (n_rows, 7)

    # ② 1차원 예외 처리 (샘플이 1행일 때)
    if preds.ndim == 1:
        preds = preds.reshape(1, -1)

    rows = []
    for (key, start_date), p in zip(keys, preds):
        for h in range(horizon):
            # (A) 파일 접두어 형식
            if test_prefix is not None:
                date_label = f"{test_prefix}+{h+1}일"
            # (B) datetime 형식
            else:
                date_label = start_date + timedelta(days=h)

            rows.append({
                '영업장명_메뉴명': key,
                '영업일자'      : date_label,
                '매출수량'      : max(0.0, float(p[h]))
            })
    return pd.DataFrame(rows)


In [31]:
from pathlib import Path, PurePath
import json, pickle

# ② 테스트 파일 수집 ------------------------------
test_path = Path(args.test)
test_files = ([test_path] if test_path.is_file()
              else sorted(test_path.glob("TEST*.csv")))
if not test_files:
    raise FileNotFoundError(f"{test_path} 에서 TEST*.csv 를 찾을 수 없습니다.")
print(f"[INFO] 예측 대상 TEST 파일 {len(test_files)}개")


pred_list = []
for p in test_files:
    tdf = add_calendar_feats(read_data(p))          # (a) 달력 피처
    X_tst, keys = build_lag_samples(tdf, train_mode=False)

    # 위치·split 파생 -------------------------------
    X_tst[['영업장명', '메뉴명']] = (
    X_tst['영업장명_메뉴명'].str.split('_', n=1, expand=True)
)
    X_tst['x'] = X_tst['영업장명'].map(lambda s: store_xy[s][0])
    X_tst['y'] = X_tst['영업장명'].map(lambda s: store_xy[s][1])
    X_tst = X_tst.drop(columns=['영업장명','메뉴명'])
    # -----------------------------------------------

    prefix  = p.stem            # TEST_00 …
    pred_df = predict_test(model, X_tst, keys, test_prefix=prefix)
    pred_list.append(pred_df)
if not pred_list:
    raise RuntimeError("예측된 결과가 없습니다. TEST 파일/데이터 확인!")

# ④ concat & 저장 -------------------------------
full_pred = (pd.concat(pred_list, ignore_index=True)
              .sort_values(['영업장명_메뉴명','영업일자']))
out_path = Path(args.out)
if out_path.is_dir():
    out_path = out_path / "submission.csv"
full_pred.to_csv(out_path, index=False)
print(f"[DONE] 최종 {full_pred.shape[0]:,}행 저장 → {out_path.resolve()}")

[INFO] 예측 대상 TEST 파일 10개


window: 100%|██████████| 193/193 [00:00<00:00, 497.31it/s]


[DONE] 최종 13,510행 저장 → /content/drive/MyDrive/data/submission.csv


In [32]:
df = pd.read_csv("/content/drive/MyDrive/data/submission.csv")
df

,영업장명_메뉴명,영업일자,매출수량
0,느티나무 셀프BBQ_1인 수저세트,TEST_00+1일,7.997457
1,느티나무 셀프BBQ_1인 수저세트,TEST_00+2일,3.074745
2,느티나무 셀프BBQ_1인 수저세트,TEST_00+3일,5.882447
3,느티나무 셀프BBQ_1인 수저세트,TEST_00+4일,6.259794
4,느티나무 셀프BBQ_1인 수저세트,TEST_00+5일,6.565423
...,...,...,...
13505,화담숲카페_현미뻥스크림,TEST_09+3일,16.235830
13506,화담숲카페_현미뻥스크림,TEST_09+4일,20.791828
13507,화담숲카페_현미뻥스크림,TEST_09+5일,9.786564
13508,화담숲카페_현미뻥스크림,TEST_09+6일,12.176413


In [33]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량']
    ))

    final_df = sample_submission.copy()

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:  # 메뉴명들
            final_df[col] = final_df[col].astype(float)
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)

    return final_df

In [34]:
sample_submission = pd.read_csv("/content/drive/MyDrive/data/baseline_submission.csv")
submission = convert_to_submission_format(df, sample_submission)
submission.to_csv(base_path+'baseline_submission_xgb_deadcut_14_xy_conti_stl.csv', index=False, encoding='utf-8-sig')

In [35]:
submission

,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,7.997457,4.329799,6.081805,3.957545,0.891883,1.877237,4.868002,2.671372,3.136456,...,6.376564,13.932940,16.253632,6.973278,59.854111,42.942753,8.607640,37.616695,9.924732,21.941065
1,TEST_00+2일,3.074745,4.457676,1.471315,0.932017,0.293153,0.742113,7.230584,1.048869,1.179974,...,2.120283,1.927096,2.422756,0.346684,6.094793,0.000000,0.178259,0.000000,1.891621,4.533075
2,TEST_00+3일,5.882447,13.591228,3.303473,1.600294,0.129798,0.753625,4.827273,1.440490,1.518089,...,3.147729,7.815788,4.561386,2.454759,29.155788,24.663342,2.347885,12.625809,5.506888,8.437785
3,TEST_00+4일,6.259794,2.974051,4.293386,2.074402,0.329455,1.381454,20.716068,3.481161,2.314548,...,2.707485,4.731916,5.473855,3.530047,23.125835,18.465141,3.019071,25.469893,4.370998,7.624175
4,TEST_00+5일,6.565423,45.840549,4.284081,2.074136,0.317629,1.623806,14.744637,4.310460,2.719090,...,2.990185,5.795126,6.163027,6.320916,19.993107,12.192832,4.402796,20.044205,6.149501,7.488283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,TEST_09+3일,0.591448,95.508224,2.257433,0.906483,0.301519,0.126391,11.410161,0.345068,0.947579,...,4.860393,10.163194,9.799626,5.072508,80.777527,14.937888,33.818115,55.596771,11.959085,16.235830
66,TEST_09+4일,0.523411,61.555367,2.216935,1.584073,0.633875,0.412326,24.343433,0.551231,0.776754,...,4.021095,7.847091,6.433663,4.007779,48.032162,12.380853,39.502193,37.466133,12.290423,20.791828
67,TEST_09+5일,2.130664,59.810471,3.623732,1.769685,0.378803,0.750097,14.608055,0.487792,1.399023,...,2.095218,7.047032,5.994805,4.320029,46.036728,15.396742,31.320358,27.856770,6.712381,9.786564
68,TEST_09+6일,5.614030,87.587357,5.231860,2.805517,1.274132,2.249567,16.247650,1.218688,2.514552,...,3.427401,12.107269,6.393444,3.817135,44.118759,5.624360,49.812347,48.916500,6.853631,12.176413
